# MLOps Workshop: Model Packaging und Containerisierung

## Einführung
In diesem Notebook lernen wir, wie wir unseren Customer Churn Predictor in einen Docker Container verpacken. Wir werden sowohl das trainierte Modell als auch die FastAPI-Anwendung containerisieren.

## Lernziele
Nach Abschluss dieses Notebooks können Sie:
- Ein ML-Modell für die Produktion vorbereiten
- Eine FastAPI-Anwendung containerisieren
- Best Practices für ML-Model-Serving implementieren
- Docker-Container für ML-Anwendungen erstellen und testen

## Prerequisites
- Abgeschlossenes Modelltraining und -evaluierung (Notebooks 03 und 04)
- Docker auf dem System installiert
- FastAPI Backend implementiert

## 1. Modell-Export und Verifikation

Zuerst laden wir unsere benötigten Bibliotheken:


In [2]:
import mlflow
import pandas as pd
import numpy as np
from mlflow.tracking import MlflowClient
import os
from pathlib import Path

# MLflow Setup
mlflow.set_tracking_uri("file:./mlruns")
client = MlflowClient()


### Aufgabe 1: Modell laden und überprüfen
Verifizieren Sie das trainierte Modell und seine Features.

<details>
<summary>👉 Lösung anzeigen</summary>

```python
# Modell laden
model_name = "customer_churn_predictor"
model_version = client.get_latest_versions(model_name, stages=["Staging"])[0]
model = mlflow.sklearn.load_model(f"runs:/{model_version.run_id}/model")

# Features überprüfen
print("Modell Features:", model.feature_names_in_)

# Beispiel-Prediction durchführen
test_data = pd.DataFrame({
    'gender': [1],
    'tenure': [12],
    'MonthlyCharges': [89.9],
    'TotalCharges': [1078.8],
    'AvgCostPerService': [29.97],
    'CustomerAge': [42]
})

prediction = model.predict_proba(test_data)
print("\nTest Prediction:", prediction[:, 1])
```
</details>


In [ ]:
# Modell laden
model_name = "customer_churn_predictor"
model_version = client.get_latest_versions(model_name, stages=["Staging"])[0]
model = mlflow.sklearn.load_model(f"runs:/{model_version.run_id}/model")

# Features überprüfen
print("Modell Features:", model.feature_names_in_)

# Beispiel-Prediction durchführen
test_data = pd.DataFrame({
    'gender': [1],
    'tenure': [12],
    'MonthlyCharges': [89.9],
    'TotalCharges': [1078.8],
    'AvgCostPerService': [29.97],
    'CustomerAge': [42]
})

prediction = model.predict_proba(test_data)
print("\nTest Prediction:", prediction[:, 1])

## 2. API-Dateien vorbereiten

### Aufgabe 2: Erstellen Sie die requirements.txt
Erstellen Sie eine requirements.txt mit allen benötigten Paketen.

<details>
<summary>👉 Lösung anzeigen</summary>

```bash
# requirements.txt
fastapi==0.104.1
uvicorn==0.24.0
pandas==2.1.3
numpy==1.26.2
scikit-learn==1.3.2
mlflow==2.8.1
pydantic==2.5.2
python-multipart==0.0.6
```
</details>


### Aufgabe 3: Projektstruktur erstellen
Erstellen Sie die benötigte Verzeichnisstruktur.

<details>
<summary>👉 Lösung anzeigen</summary>

```bash
project_root/
├── src/
│   └── api/
│       ├── main.py
│       └── test_main.py
├── models/
│   └── model/
├── notebooks/
│   └── mlruns/
├── Dockerfile
├── docker-compose.yml
└── requirements.txt
```
</details>


## 3. Docker Configuration

### Aufgabe 4: Dockerfile erstellen
Erstellen Sie ein Dockerfile für die FastAPI-Anwendung.

<details>
<summary>👉 Lösung anzeigen</summary>

```dockerfile
# Base image
FROM python:3.9-slim

# Set working directory
WORKDIR /app

# Copy requirements first for better caching
COPY requirements.txt .

# Install dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy the rest of the application
COPY src/api /app/api
COPY notebooks/mlruns /app/mlruns

# Environment variables
ENV PYTHONPATH=/app
ENV MODEL_PATH=/app/mlruns

# Switch to non-root user
RUN useradd -m appuser
USER appuser

# Command to run the application
CMD ["uvicorn", "api.main:app", "--host", "0.0.0.0", "--port", "8000"]
```
</details>


### Aufgabe 5: Docker Compose erstellen
Erstellen Sie eine docker-compose.yml für einfaches Deployment.

<details>
<summary>👉 Lösung anzeigen</summary>

```yaml
version: '3.8'

services:
  churn-predictor:
    build: .
    ports:
      - "8000:8000"
    volumes:
      - ./notebooks/mlruns:/app/mlruns
    environment:
      - MODEL_PATH=/app/mlruns
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
```
</details>


## 4. Container Build und Test

### Aufgabe 6: Container erstellen und testen
Bauen und testen Sie den Docker Container.

<details>
<summary>👉 Lösung anzeigen</summary>

```bash
# Container bauen
docker build -t churn-predictor:latest .

# Container starten
docker run -p 8000:8000 churn-predictor:latest

# Test mit curl
curl -X 'POST' \
  'http://localhost:8000/predict' \
  -H 'Content-Type: application/json' \
  -d '{
    "data": [{
        "gender": 1,
        "tenure": 12,
        "MonthlyCharges": 89.9,
        "TotalCharges": 1078.8,
        "AvgCostPerService": 29.97,
        "CustomerAge": 42
    }]
}'
```

Python-Code zum Testen:
```python
import requests
import json

# Test-Daten
test_data = {
    "data": [{
        "gender": 1,
        "tenure": 12,
        "MonthlyCharges": 89.9,
        "TotalCharges": 1078.8,
        "AvgCostPerService": 29.97,
        "CustomerAge": 42
    }]
}

# API-Request
response = requests.post(
    'http://localhost:8000/predict',
    json=test_data
)

print("Status Code:", response.status_code)
print("Response:", response.json())
```
</details>


## 5. Performance Testing

### Aufgabe 7: Last- und Performance-Tests
Implementieren Sie einfache Last-Tests.

<details>
<summary>👉 Lösung anzeigen</summary>

```python
import time
import concurrent.futures
import statistics

def make_prediction_request():
    """Einzelne Prediction durchführen"""
    response = requests.post(
        'http://localhost:8000/predict',
        json=test_data
    )
    return response.elapsed.total_seconds()

def run_load_test(num_requests=100, concurrent_requests=10):
    """Load Test durchführen"""
    response_times = []
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=concurrent_requests) as executor:
        futures = [executor.submit(make_prediction_request) for _ in range(num_requests)]
        for future in concurrent.futures.as_completed(futures):
            response_times.append(future.result())
    
    print(f"\nLoad Test Results (n={num_requests}):")
    print(f"Mean response time: {statistics.mean(response_times):.3f}s")
    print(f"Median response time: {statistics.median(response_times):.3f}s")
    print(f"95th percentile: {np.percentile(response_times, 95):.3f}s")

# Load Test ausführen
run_load_test()
```
</details>



## Hausaufgaben und weiterführende Übungen
1. Implementieren Sie ein Multi-Stage Docker Build
2. Fügen Sie Prometheus Metriken hinzu
3. Implementieren Sie ein Blue-Green Deployment
4. Erstellen Sie ein Kubernetes Deployment Manifest
5. Implementieren Sie automatische Skalierung

## Nächste Schritte
- Deployment in einer Cloud-Umgebung
- Implementierung von Monitoring und Logging
- CI/CD Pipeline aufsetzen
- Kubernetes Deployment vorbereiten

## Hilfreiche Befehle

```bash
# Docker
docker build -t churn-predictor .
docker run -p 8000:8000 churn-predictor
docker ps
docker logs <container_id>

# Docker Compose
docker-compose up -d
docker-compose logs
docker-compose down

# Curl Tests
curl http://localhost:8000/health
curl http://localhost:8000/model-info
```

## Zusätzliche Hinweise
- Verwenden Sie immer spezifische Versionen in requirements.txt
- Implementieren Sie Health Checks für Container Orchestration
- Nutzen Sie Docker Layer Caching effektiv
- Minimieren Sie die Image-Größe
- Beachten Sie Security Best Practices